# 信息的定量描述
直观理解： 
若消息发生的概率很大，受信者事先已经有所估计，则该消息的信息量就很小。 
若消息发生的概率很小，受信者感觉到很突然，该消息所含有的信息量就很大。 
所以信息量和概率联系在了一起，信息量可以表示为概率的函数。那么怎样的函数可以用来描述信息量呢？函数f(p)f(p)应该满足以下条件： 
1. f(p)应该是概率p的严格单调递减函数， 
2. 当p=1时，f(p)=0 
3. 当p=0时，f(p)=∞
4. 两个独立事件的联合信息量应该等于它们信息量之和。 
以下是f(p)=−log(p)f(p)=−log(p)的图像，满足以上的所有的要求。

# ID3算法熵计算公式

1. 选择分类\\(x_i\\)的概率

2. 信息期望值

自信息和熵的定义
若一个消息xx出现的概率为p，那么这个消息所含有的信息量为 
$$I=−log(p)$$

上式称为消息x的自信息，自信息有两种含义： 
1. 当该消息发生之前，表示发生该消息的不确定性， 
2. 当该消息发生之后，表示消息所含有的信息量。 
信源含有的信息量是信源发出的所有可能消息的平均不确定性，香农把信源所含有的信息量称为熵，是指每个符号所含有的信息量（自信息）的统计平均。若X是一个离散随机变量，概率分布为\\(p(x)=P(X=x)\\),\\(x∈X\\)，那么XX的熵为 
$$H(X)=\sum_{i}^{N}p(xi)I(xi)=−\sum_{i}^{N}p(xi)logp(xi)$$


$$H(X)=-\sum_{i=1}^{n}p(x_i)log_2p(x_i)$$


# 设计决策树
| 依水 | 鳍否 | 鱼否 |
| ---- | ---- | ---- |
| yes  | yes  | yes  |
| yes  | yes  | yes  |
| yes  | no  | no   |
| no  | yes  | no   |
| no  | no  | no   |

1. 从目的导向，先看最终决策项的信息熵 \\(H(X)\\)  --求先验熵，即求**鱼否**的信息熵。
$$H(鱼)=-\frac{2}{5}log_2\frac{2}{5}$$
$$H(非鱼)=-\frac{3}{5}log_2\frac{3}{5}$$
$$H(鱼判别)=H(鱼)+H(非鱼)$$

2. 已处于某条件类下的各项，--计算后验熵，即求条件中每项成立的各个熵
$$H(鱼否|鳍)=-\frac{2}{3}log_2\frac{2}{3}-\frac{1}{3}log_2\frac{1}{3}$$
$$H(鱼否|无鳍)=-\frac{0}{2}log_2\frac{0}{2}-\frac{2}{2}log_2\frac{2}{2}$$
$$H(鱼否|依水)=-\frac{2}{3}log_2\frac{2}{3}-\frac{1}{3}log_2\frac{1}{3}$$
$$H(鱼否|非水)=-\frac{0}{2}log_2\frac{0}{2}-\frac{2}{2}log_2\frac{2}{2}$$

3. 对给定\\(Y\\)成立的条件下，\\(X\\)集合求和后验熵即条件熵\\(H(X)\\)，即求 \\(H(鱼否|决策项)\\)，即条件熵=\\(p_i\\)*先验熵
$$H(鱼否|鳍否)=H(鱼否|鳍)+H(鱼否|无鳍)$$
$$H(鱼否|依水否)=H(鱼否|依水)+H(鱼否|非水)$$

4. 信息增益，对**每组判别项与是否结果**的关系
$$I(鱼否：鳍否)=H(鱼否)-H(鱼否|鳍否)$$
$$I(鱼否：依水否)=H(鱼否)-H(鱼否|依水否)$$

先计算先验熵，再计算后验熵，最后计算条件熵


按不确定性最小，信息量最大排在树根
即，信息增益\\(I\\)最大的作为树根


In [2]:
from math import log
def calcShannonEnt(dataSet): #这个数据集计算香农熵格式要求最后一列数据为判别标签作为H(Label)
  numEntries = len(dataSet) #数据集长度
  labelCounts = {}
  for featVec in dataSet: #对数据集内所有判别式初始化为0
    currentLabel = featVec[-1] #区每组数据尾部作为最终判别标签
    if currentLabel not in labelCounts.keys():
      labelCounts[currentLabel] = 0
    labelCounts[currentLabel] += 1
  shannonEnt = 0.0 #香农熵
  for key in labelCounts:#每个类标签出现概率之和，先验熵
    prob = float(labelCounts[key])/numEntries  #计算每个类标签的出现概率
    shannonEnt -= prob * log(prob,2) #后验熵 += 后验熵
  return shannonEnt #先验熵

In [105]:
def createDataSet():
  dataSet = [
    [1,1,'yes'],
    [1,1,'yes'],
    [1,0,'no'],
    [0,1,'no'],
    [0,1,'no']
  ]
  labels = ['no surfacing','flippers']
  return dataSet,labels

# 按照给定特征划分数据集

In [106]:
def splitDataSet(dataSet,index,value):#其实这个东西就是用作统计概率用的分数项
  retDataSet = []
  numFeatures = len(dataSet[0]) - 1
  for featVec in dataSet:
    if featVec[index] == value:
      retDataSet.append([featVec[index],featVec[numFeatures]]) #对应项，对应结果
  return retDataSet

In [114]:
myDat,labels = createDataSet()
print(myDat)
sn = calcShannonEnt(myDat)
print(sn)
x1 = splitDataSet(myDat,0,1)
x2 = splitDataSet(myDat,0,0)
x3 = splitDataSet(myDat,1,1)
x4 = splitDataSet(myDat,1,0)
print(x1,'\n',x2)
print(x3,'\n',x4)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0.9709505944546686
[[1, 'yes'], [1, 'yes'], [1, 'no']] 
 [[0, 'no'], [0, 'no']]
[[1, 'yes'], [1, 'yes'], [1, 'no'], [1, 'no']] 
 [[0, 'no']]


# 选择最好的数据集划分方式

In [121]:
def chooseBestFeatureToSplit(dataSet): #要求最后一列为数据为判别标签作为H(Label)
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet) #即得到H(结果)
    bestInfoGain, bestFeature = 0.0,-1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet] #判别项 列表值
        uniqueVals = set(featList) #取判别项唯一值
        newEntropy = 0.0
        for value in uniqueVals :#求信息增益
            subDataSet = splitDataSet(dataSet,i,value) #对每一列判别项，取每一组值分类
            prob = len(subDataSet)/float(len(dataSet)) #求每一组值发生概率
            newEntropy += prob * calcShannonEnt(subDataSet) #条件熵 +=条件熵
        infoGain = baseEntropy - newEntropy
        print('infoGain=',infoGain,'bestFeature=',i,baseEntropy,newEntropy)
        if (infoGain > bestInfoGain): #求信息增益最大项
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [110]:
def majorityCnt(classList): #返回类中最多项的次数
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
            classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

# 创建树

In [111]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree
    

In [112]:
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueofFeat = secondDict[key]
    print('+++',firstStr,'xxx',secondDict,'---',key,'>>>',valueofFeat)
    if isinstance(valueofFeat,dict):
        classLabel = classify(valueofFeat,featLabels,testVec)
    else:
        classLabel = valueofFeat
    return  classLabel

In [113]:
myTree = createTree(myDat,labels)

infoGain= 0.4199730940219749 bestFeature= 0 0.9709505944546686 0.5509775004326937
infoGain= 0.17095059445466854 bestFeature= 1 0.9709505944546686 0.8
infoGain= 0.0 bestFeature= 0 0.9182958340544896 0.9182958340544896
